## Daten zusammenfügen und erster Blick auf die Daten

In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Datensatz einlesen
umsatzdaten = pd.read_csv('0_DataPreparation/umsatzdaten_gekuerzt.csv')
wetter = pd.read_csv('0_DataPreparation/wetter.csv')
kiwo = pd.read_csv('0_DataPreparation/kiwo.csv')

# Daten zusammenführen als outer join
merged_data = pd.merge(umsatzdaten, wetter, on='Datum', how='outer')
merged_data = pd.merge(merged_data, kiwo, on='Datum', how='outer')

# Umsatzdaten anzeigen
print(merged_data.head())
print(merged_data.info())

# Deskriptive Statistik für Umsatzdaten
print(merged_data.describe())
print(merged_data.isnull())



        Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0  2012-01-01          NaN     NaN         8.0      9.8250   
1  2012-01-02          NaN     NaN         7.0      7.4375   
2  2012-01-03          NaN     NaN         8.0      5.5375   
3  2012-01-04          NaN     NaN         4.0      5.6875   
4  2012-01-05          NaN     NaN         6.0      5.3000   

   Windgeschwindigkeit  Wettercode  KielerWoche  
0                 14.0        58.0          NaN  
1                 12.0         NaN          NaN  
2                 18.0        63.0          NaN  
3                 19.0        80.0          NaN  
4                 23.0        80.0          NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10119 entries, 0 to 10118
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Datum                10119 non-null  object 
 1   Warengruppe          9334 non-null   float64
 2   Umsatz        

## Temperatur in kategoriale Variable transformieren und Feiertage hinzufügen

In [89]:
# Temperatur in kategoriale Variable umwandeln (Kategorien numerisch kodiert)
merged_data['Temperatur_kat'] = pd.cut(merged_data['Temperatur'], bins=[-10, 0, 20, 45], labels=[0, 1, 2])


In [90]:
import pandas as pd
import holidays

# Liste der Feiertage für Deutschland
de_holidays = holidays.Germany()

# Datumsbereich
start_date = '2012-01-01'
end_date = '2019-08-01'

# Liste aller Tage im Datumsbereich
date_range = pd.date_range(start=start_date, end=end_date)

# Liste zur Speicherung des Feiertagsstatus
holiday_status = []

# Überprüfung jedes Datums auf Feiertag
for date in date_range:
    if date in de_holidays:
        holiday_status.append(1)  # Feiertag
    else:
        holiday_status.append(0)  # normaler Tag

# Erstellen eines DataFrame mit Datum und Feiertagsstatus
holiday_df = pd.DataFrame({'Datum': date_range, 'Feiertag': holiday_status})

# Anzeigen der ersten Zeilen des DataFrame
print(holiday_df.head())

# Datum in umsatzdaten in datetime umwandeln
merged_data['Datum'] = pd.to_datetime(merged_data['Datum'])

# holiday_df mit umsatzdaten zusammenführen
merged_data = pd.merge(merged_data, holiday_df, on='Datum', how='outer')

# Anzeigen der ersten Zeilen des DataFrame
print(merged_data)

       Datum  Feiertag
0 2012-01-01         1
1 2012-01-02         0
2 2012-01-03         0
3 2012-01-04         0
4 2012-01-05         0
           Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0     2012-01-01          NaN     NaN         8.0      9.8250   
1     2012-01-02          NaN     NaN         7.0      7.4375   
2     2012-01-03          NaN     NaN         8.0      5.5375   
3     2012-01-04          NaN     NaN         4.0      5.6875   
4     2012-01-05          NaN     NaN         6.0      5.3000   
...          ...          ...     ...         ...         ...   
10280 2019-07-28          NaN     NaN         3.0     23.3500   
10281 2019-07-29          NaN     NaN         6.0     25.2500   
10282 2019-07-30          NaN     NaN         7.0     20.7375   
10283 2019-07-31          NaN     NaN         6.0     20.4500   
10284 2019-08-01          NaN     NaN         5.0     21.0625   

       Windgeschwindigkeit  Wettercode  KielerWoche Temperatur_kat  Feiertag  
0 

## Variable Wochenende hinzufügen

In [91]:
merged_data['Datum'] = pd.to_datetime(merged_data['Datum'])
merged_data['Wochenende'] = merged_data['Datum'].dt.dayofweek >= 5


## Variable Temperaturabweichung vom Monatsmittel hinzufügen und binäre Variablen numerisch kodieren

In [92]:
# Variable für Temperaturabweichung um Mittelwert der Temperatur für jeden Monat berechnen
# Um die Monate zu extrahieren
merged_data['Monat'] = merged_data['Datum'].dt.month

# Mittelwerte der Temperatur für jeden Monat berechnen
monatsmittel_temp = merged_data.groupby('Monat')['Temperatur'].transform('mean')

# Um die Temperaturabweichung zu berechnen
merged_data['Temp_abw'] = (merged_data['Temperatur'] - monatsmittel_temp).abs() > 5

# Kieler Woche, Wochenende und Temp.abw binär mit 0 und 1 kodieren
merged_data['KielerWoche'] = merged_data['KielerWoche'].fillna(0).astype(int)
merged_data['Wochenende'] = merged_data['Wochenende'].fillna(0).astype(int)
merged_data['Temp_abw'] = merged_data['Temp_abw'].fillna(0).astype(int)

# Anzeigen der ersten Zeilen des DataFrame mit der neuen Spalte
print(merged_data.head())

# Wie oft True für Temp_abw
print(merged_data['Temp_abw'].value_counts())


       Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0 2012-01-01          NaN     NaN         8.0      9.8250   
1 2012-01-02          NaN     NaN         7.0      7.4375   
2 2012-01-03          NaN     NaN         8.0      5.5375   
3 2012-01-04          NaN     NaN         4.0      5.6875   
4 2012-01-05          NaN     NaN         6.0      5.3000   

   Windgeschwindigkeit  Wettercode  KielerWoche Temperatur_kat  Feiertag  \
0                 14.0        58.0            0              1         1   
1                 12.0         NaN            0              1         0   
2                 18.0        63.0            0              1         0   
3                 19.0        80.0            0              1         0   
4                 23.0        80.0            0              1         0   

   Wochenende  Monat  Temp_abw  
0           1      1         1  
1           0      1         0  
2           0      1         0  
3           0      1         0  
4          

## Bewölkung und Windgeschwindigkeit in kategoriale Variable transformieren

In [93]:
# Variable erstellen, die die Bewölkung in kategoriale Variable umwandelt
merged_data['Bewoelkung_kat'] = pd.cut(merged_data['Bewoelkung'], bins=[0, 3, 6, 9], labels=[0, 1, 2])

# Anzeigen der ersten Zeilen des DataFrame mit der neuen Spalte
print(merged_data.head())

# Nur minimal größerer Zusammenhang zwischen Bewölkung und Umsatz (siehe Korrelation)

# Windgeschwindigkeit in kategoriale Variable (minimum 3 und mximum 35) umwandeln
merged_data['Windgeschwindigkeit_kat'] = pd.cut(merged_data['Windgeschwindigkeit'], bins=[0, 5, 10, 15, 20, 25, 30, 35], labels=[0, 1, 2, 3, 4, 5, 6])

# Anzeigen der ersten Zeilen des DataFrame mit der neuen Spalte
print(merged_data.head())
print(merged_data.isna().sum())

       Datum  Warengruppe  Umsatz  Bewoelkung  Temperatur  \
0 2012-01-01          NaN     NaN         8.0      9.8250   
1 2012-01-02          NaN     NaN         7.0      7.4375   
2 2012-01-03          NaN     NaN         8.0      5.5375   
3 2012-01-04          NaN     NaN         4.0      5.6875   
4 2012-01-05          NaN     NaN         6.0      5.3000   

   Windgeschwindigkeit  Wettercode  KielerWoche Temperatur_kat  Feiertag  \
0                 14.0        58.0            0              1         1   
1                 12.0         NaN            0              1         0   
2                 18.0        63.0            0              1         0   
3                 19.0        80.0            0              1         0   
4                 23.0        80.0            0              1         0   

   Wochenende  Monat  Temp_abw Bewoelkung_kat  
0           1      1         1              2  
1           0      1         0              2  
2           0      1         0  

## Zeitvariablen und Lagged Variables hinzufügen

In [94]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

# Neue Zeitvariablen hinzufügen
merged_data['Wochentag'] = pd.to_datetime(merged_data['Datum']).dt.dayofweek
merged_data['Kalenderwoche'] = pd.to_datetime(merged_data['Datum']).dt.isocalendar().week
merged_data['Jahreszeit'] = pd.to_datetime(merged_data['Datum']).dt.month % 12 // 3 + 1

# Lagged Variables hinzufügen (Umsatz einen Tag und eine Woche zuvor)
merged_data['Umsatz_lag1'] = merged_data['Umsatz'].shift(1)
merged_data['Umsatz_lag7'] = merged_data['Umsatz'].shift(7)

# Anzeigen von random Zeilen des DataFrame mit den neuen Spalten
print(merged_data.sample(5))
print(merged_data)


          Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
5747 2016-04-24          3.0  153.324122         4.0      5.7500   
9338 2018-04-05          2.0  315.005856         7.0      8.6625   
9915 2018-07-31          2.0  586.081666         5.0     31.2875   
8071 2017-08-01          2.0  557.470898         6.0     21.3750   
7257 2017-02-17          1.0  151.638374         7.0      6.0000   

      Windgeschwindigkeit  Wettercode  KielerWoche Temperatur_kat  Feiertag  \
5747                 10.0        21.0            0              1         0   
9338                 19.0        61.0            0              1         0   
9915                 11.0         NaN            0              2         0   
8071                  7.0        21.0            0              2         0   
7257                  9.0        61.0            0              1         0   

      Wochenende  Monat  Temp_abw Bewoelkung_kat Windgeschwindigkeit_kat  \
5747           1      4         1       

## Korrelationen

In [95]:
# Korrelationsmatrix für Umsatzdaten
print(merged_data.corr())


                            Datum  Warengruppe    Umsatz  Bewoelkung  \
Datum                    1.000000    -0.006639 -0.086901   -0.019253   
Warengruppe             -0.006639     1.000000 -0.062589    0.023795   
Umsatz                  -0.086901    -0.062589  1.000000   -0.089916   
Bewoelkung              -0.019253     0.023795 -0.089916    1.000000   
Temperatur              -0.019917    -0.047015  0.222935   -0.383888   
Windgeschwindigkeit     -0.020052    -0.009078  0.014705    0.042813   
Wettercode              -0.114740     0.009547 -0.010091    0.525852   
KielerWoche              0.033931    -0.010207  0.058244    0.010423   
Temperatur_kat           0.027973    -0.022894  0.150435   -0.354348   
Feiertag                 0.000659    -0.006163  0.043574   -0.031793   
Wochenende               0.002347     0.000457  0.163097   -0.006312   
Monat                   -0.044090     0.080817  0.013889    0.032380   
Temp_abw                 0.029410    -0.000134 -0.007693   -0.09

## Lineares Modell ohne Interaktionseffekte

In [103]:
# Fit the linear model (erster Versuch)
mod = smf.ols('Umsatz ~ C(Warengruppe) + C(Bewoelkung_kat) + Temperatur + C(KielerWoche) + C(Feiertag) + C(Wochenende) + C(Wochentag) + C(Jahreszeit) + Umsatz_lag1 + Umsatz_lag7', data=merged_data).fit()

# Print the summary
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.722
Method:                 Least Squares   F-statistic:                     989.8
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        15:06:48   Log-Likelihood:                -45891.
No. Observations:                7992   AIC:                         9.183e+04
Df Residuals:                    7970   BIC:                         9.198e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 43

## Lineares Modell mit Interaktionseffekten

In [97]:
# Modell mit Interaktionseffekten
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
import itertools

# Formel für die multiple lineare Regression mit ausgewählten Interaktionseffekten
formula = 'Umsatz ~ C(Warengruppe) + Temperatur + Wettercode + C(KielerWoche) + C(Feiertag) + C(Wochenende) + Monat'

# Hinzufügen spezifischer Interaktionseffekte
interaction_terms = [
    'Temperatur:Wochenende',
    'KielerWoche:Warengruppe',
    'Wettercode:Temperatur',
    'Monat:Temperatur'
]

# Hinzufügen der Interaktionseffekte zur Formel
for term in interaction_terms:
    formula += ' + ' + term

# Modell anpassen
model = smf.ols(formula=formula, data=merged_data).fit()

# Zusammenfassung des Modells ausgeben
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     1113.
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:53:49   Log-Likelihood:                -40330.
No. Observations:                7009   AIC:                         8.069e+04
Df Residuals:                    6993   BIC:                         8.080e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

## Test mit Ninas Modellen

- Ich finde nicht wirklich eine Erklärung, warum wir so unterschiedliche Ergebnisse bekommen
- Das erste Modell ist besser als Ninas Modell hier; vielleicht sollten wir das auch noch versuchen

In [98]:
mod = smf.ols('Umsatz ~ C(Warengruppe) + Temperatur + Wochenende + KielerWoche  + C(Wochentag)', data=merged_data).fit()

# Print the summary
print(mod.summary())


                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     1775.
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        14:53:50   Log-Likelihood:                -53757.
No. Observations:                9318   AIC:                         1.075e+05
Df Residuals:                    9304   BIC:                         1.076e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                60.29

## Daten aufteilen und Vorhersagen testen

In [99]:
# Umsatzdaten aufteilen
import pandas as pd
from sklearn.model_selection import train_test_split
# Set a random seed for reproducibility
random_state = 42
# First, split the data into training (70%) and remaining (30%)
train_data, remaining_data = train_test_split(merged_data, train_size=0.7, random_state=random_state)
# Now split the remaining data into validation (2/3 of remaining) and test (1/3 of remaining)
validation_data, test_data = train_test_split(remaining_data, test_size=0.3333, random_state=random_state)
# Check the dimensions of the datasets
print("Training dataset dimensions:", train_data.shape)
print("Validation dataset dimensions:", validation_data.shape)
print("Test dataset dimensions:", test_data.shape)


Training dataset dimensions: (7199, 20)
Validation dataset dimensions: (2057, 20)
Test dataset dimensions: (1029, 20)
